In [1]:
import os

from IPython.display import display, Markdown

In [2]:
REPO_PATH = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))
TASK_PATH = os.path.join(REPO_PATH, "tasks", "02-structural-linguistics")
DATA_PATH = os.path.join(TASK_PATH, "data")

In [3]:
def show_markdown(path):
    with open(path, 'r') as md:
        content = md.read()
    display(Markdown(content))

In [4]:
show_markdown(os.path.join(TASK_PATH, "2-headlines.md"))

## Заголовки новин

### 1. Форматування

[The Associated Press Stylebook](https://www.amazon.com/Associated-Press-Stylebook-2017-Briefing/dp/0465093043/) - це посібник зі стилю, яким часто послуговуються журналісти по всьому світу. Він рекомендує такі правила форматування заголовків:
1. З великої літери потрібно писати слова довжиною 4 чи більше літер.
2. З великої літери потрібно писати перше і останнє слово заголовку, незалежно від частини мови.
3. З великої літери потрібно писати іменники, займенники, дієслова, прикметники, прислівники та підрядні сполучники.
4. Якщо слово написане через дефіс, велику літеру потрібно додати для кожної частинки слова (наприклад, правильно "Self-Reflection", а не "Self-reflection").
5. З маленької літери потрібно писати всі інші частини мови: артиклі/визначники, сурядні сполучники, прийменники, частки, вигуки.

**Завдання:**
1. напишіть програму, яка форматує заголовки за вказаними правилами
2. перевірте якість роботи програми на [валідаційній вибірці](data/headlines-test-set.json)
3. проженіть вашу програму на [корпусі заголовків з The Examiner](data/examiner-headlines.txt) і вирахуйте, скільки заголовків там відформатовано за правилами (скільки заголовків залишились незмінними після форматування)
4. збережіть програму та числові результати у директорії з вашим іменем

Якщо потрібно продебажити роботу програми, робіть це на [корпусі заголовків з The Examiner](data/examiner-headlines.txt), а не на валідаційній вибірці. Спробуйте досягти хоча б 90% якості на валідаційній вибірці. Якість рахуємо за повним збігом відформатованого заголовка.

Підказка: ваша програма повинна правильно розрізняти прийменники та підрядні сполучники. Наприклад, `Do as you want` => `Do As You Want` (бо "as" тут є сполучником), but `How to use a Macbook as a table` => `How to Use a Macbook as a Table` (бо "as" тут є прийменником).

### 2. Вірусні новини

У статті [Automatic Extraction of News Values from Headline Text](http://www.aclweb.org/anthology/E17-4007) описано основні ознаки заголовків, які кидаються в очі і змушують читача таки прочитати новину:
1. іменовані сутності (імена людей, назви компаній тощо)
2. емоційне забарвлення
3. вищий і найвищий ступені порівняння
4. близькість до читача
5. елемент несподіванки
6. унікальність

**Завдання:**
1. Напишіть програму, яка аналізує заголовок за першими трьома ознаками (у спрощеній формі)
   * Чи є в заголовку іменовані стуності?
   * Чи є заголовок позитивно чи негативно забарвлений?
   * Чи є в заголовку прикметники та прислівники вищого і найвищого ступенів порівняння?
2. Проженіть вашу програму на [корпусі заголовків з The Examiner](data/examiner-headlines.txt). Для кожної з трьох ознак визначте відсоток заголовків у корпусі, які її мають.
3. Збережіть програму та пораховану статистику в директорії з вашим іменем.

Додаткова інформація:
- Типи сутностей, які впливають на "вірусність" заголовка, виберіть самостійно.
- Для визначення емоційного забарвлення, використайте [SentiWordNet](https://github.com/aesuli/sentiwordnet). Наприклад, можна перевірити, що середнє значення позитивності/негативності слова у заголовку перевищує 0.5. Для визначення середнього значення можна брати до п'яти перших значень слова з такою частиною мови. Будьте креативними та експериментуйте. Можна користуватися SentiWordNet з бібліотеки [NLTK](http://www.nltk.org/howto/sentiwordnet.html).

### Джерела

Ви можете використати будь-яку мову програмування та будь-яку NLP-бібліотеку.

Набір заголовків взятий із https://www.kaggle.com/therohk/examine-the-examiner.


In [5]:
import re
import spacy

import pandas as pd
import numpy as np

In [6]:
nlp = spacy.load("en_core_web_md")

In [7]:
val_df = pd.read_table(os.path.join(DATA_PATH, "examiner-headlines.txt"), names=["original"])

print(val_df.shape)
val_df.head()

(5000, 1)


,original
0,Halep enters Rogers Cup final in straight sets...
1,The phantoms of St. Mary's
2,Talladega turmoil could spell trouble for NASC...
3,Burn those calories! Try the Very Steep Trail.
4,It's the end of the world... and I feel fine


In [8]:
test_df = pd.read_json(os.path.join(DATA_PATH, "headlines-test-set.json"))
test_df.columns = ["true", "test"]

print(test_df.shape)
test_df.head()

(100, 2)


,true,test
0,How To Design A College Curriculum to Help You...,How to Design a College Curriculum to Help You...
1,This is why you should hate Battlefield 3,This Is Why You Should Hate Battlefield 3
2,How to photograph tonight's Lyrid Meteor Shower,How to Photograph Tonight's Lyrid Meteor Shower
3,Teresa Giudice broke: 'RHONJ' star can't even ...,Teresa Giudice Broke: 'RHONJ' Star Can't Even ...
4,Murder Mystery Dinner Theater - 'Who Killed Cu...,Murder Mystery Dinner Theater - 'Who Killed Cu...


In [9]:
test_df.sample().values

array([['Craigslist advises how to deliver; seller robbed following advice',
        'Craigslist Advises How to Deliver; Seller Robbed Following Advice']],
      dtype=object)

### Formatting

In [10]:
# articles = ["a", "an", "the"]
# particles = ["not"]


def format_title(text, prnt=False):
    tokens = nlp(text)
    new_tokens = [token.text for token in tokens]
    
    if prnt:
        for item in [(token.text, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop) for token in tokens]:
            print(item)
    
    res = []
    skip = False
    for index, token in enumerate(tokens):
        if skip:
            skip = False
            continue
            
#         if token.pos_ == "PROPN":
#             if len(re.findall(r"[A-Z]+", token.text)) == 0:
#                 new_tokens[index] = token.text.capitalize()
#             else:
#                 new_tokens[index] = token.text

        if len(re.findall(r"[A-Z]+", token.text)) > 0:
            new_tokens[index] = token.text
            continue
                
        elif token.pos_ in ["NOUN", "PRON", "VERB", "ADV", "ADJ", "NUM"]:
            new_tokens[index] = token.text.capitalize()
            
        elif token.pos_ == 'DET' and token.dep_ in ("poss", "appos"):
            new_tokens[index] = token.text.capitalize()
        
        elif token.pos_ == 'SCONJ' and token.dep_ == "mark":
            new_tokens[index] = token.text.capitalize()
            
#         elif token.pos_ == 'PART' and token.tag_ == 'RB' and token.dep_ == "neg":
#             new_tokens[index] = token.text.capitalize()
        elif token.text.lower() == 'not':
            new_tokens[index] = token.text.capitalize()
            
        elif token.tag_.startswith("VB"):
            new_tokens[index] = token.text.capitalize()
                
        elif len(token.text) > 3:
            new_tokens[index] = token.text.capitalize()
            
        elif token.text == "-":
            new_tokens[index-1] = tokens[index-1].text.capitalize()
            new_tokens[index+1] = tokens[index+1].text.capitalize()
            skip = True
            
        else:
            new_tokens[index] = token.text
    
    new_tokens = [token + whitespace for token, whitespace in zip(new_tokens, [token.whitespace_ for token in tokens])]
    new_tokens = "".join(new_tokens)
    
    new_tokens = new_tokens.split(" ")
    new_tokens[0] = new_tokens[0].capitalize() if len(re.findall(r"[A-Z]+", new_tokens[0])) == 0 else new_tokens[0]
    new_tokens[-1] = new_tokens[-1].capitalize() if len(re.findall(r"[A-Z]+", new_tokens[-1])) == 0 else new_tokens[-1]

    return " ".join(new_tokens)

In [11]:
%%time

test_df["pred"] = test_df["true"].map(format_title)
print("Formatting accuracy, 100:", test_df.loc[test_df["pred"] == test_df["test"]].shape[0] / test_df.shape[0], "\n")

Formatting accuracy, 100: 0.96 

CPU times: user 577 ms, sys: 90 µs, total: 577 ms
Wall time: 576 ms


In [12]:
%%time

val_df["formatted"] = val_df["original"].map(format_title)
print("Formatted by rules, 5000:", val_df.loc[val_df["original"] == val_df["formatted"]].shape[0] / val_df.shape[0], "\n")

Formatted by rules, 5000: 0.1482 

CPU times: user 26.5 s, sys: 16.2 ms, total: 26.5 s
Wall time: 26.5 s


In [13]:
for item in val_df.sample(5).values:
    print(f"Original:\t {item[0]}")
    print(f"Formatted:\t {format_title(item[0])}", "\n")

Original:	 Former Jets kicker Jay Feely signs with Arizona Cardinals
Formatted:	 Former Jets Kicker Jay Feely Signs With Arizona Cardinals 

Original:	 Art in Denver: O'Keefe exhibit soon gone
Formatted:	 Art in Denver: O'Keefe Exhibit Soon Gone 

Original:	 'Dr. Horrible' update
Formatted:	 'Dr. Horrible' Update 

Original:	 Matt Lauer and NBC's 'Today' pay tribute to Elizabeth Edwards (video)
Formatted:	 Matt Lauer and NBC's 'Today' Pay Tribute to Elizabeth Edwards (Video) 

Original:	 Network externalities in the MMO community.
Formatted:	 Network Externalities in the MMO Community. 



### Viral News

In [14]:
from textblob import TextBlob
from nltk.corpus import sentiwordnet as swn

In [15]:
senti_df = pd.read_csv("SentiWordNet_3.0.0.txt", sep="\t")

In [16]:
senti_df['SynsetTermsListed'] = senti_df['SynsetTerms'].map(lambda x: re.findall(r"([\w\-]+)#", x))

In [17]:
senti_map = {}

for pos, words, poss, neg in senti_df[["POS", "SynsetTermsListed", "PosScore", "NegScore"]].values:
    for word in words:
        senti_map[pos + "_" + word] = senti_map.get(pos + "_" + word, []) + [(poss, neg)]

In [18]:
senti_map = {k: tuple(np.mean(np.array(v), axis=0)) for k, v in senti_map.items()}

In [19]:
pos_mapping = {"NOUN": "n", "PROPN": "n", "ADJ": "a", "ADV": "r", "VERB": "v"}

In [20]:
def check_viral(title, swn=True):
    doc = nlp(title)
    
    entities = [ent.text for ent in doc.ents]
    if not swn:
        sentiment = TextBlob(title).sentiment.polarity
    else:
        sentiment = []
    comp_degrees = []
    
    for index, token in enumerate(doc):
        if swn:
            sentiment.append(senti_map.get(pos_mapping.get(token.pos_, "") + "_" + token.text, (0, 0)))
        if token.pos_ in ("ADJ", "ADV"):
            if token.text.endswith(("er", "est")):
                comp_degrees.append(token.text)
            elif doc[index-1].text in ("more", "most"):
                comp_degrees.append(doc[index-1].text + doc[index-1].whitespace_ + token.text)
            else:
                continue
    if swn:
        pos, neg = tuple(np.mean(np.array(sentiment), axis=0))
        sentiment = pos - neg
    return entities, sentiment, comp_degrees

In [21]:
%%time

res = list(map(lambda x: check_viral(x), val_df.original.values))

CPU times: user 24.6 s, sys: 27 µs, total: 24.6 s
Wall time: 24.6 s


In [22]:
print(f"Percentage of titles with NER: {len(list(filter(lambda x: len(x[0])> 0, res))) / len(res)*100:.2f}%")
print()
print(f"Percentage of titles with positive sentiment: {len(list(filter(lambda x: x[1] > 0, res))) / len(res)*100:.2f}%")
print(f"Percentage of titles with negative sentiment: {len(list(filter(lambda x: x[1] < 0, res))) / len(res)*100:.2f}%")
print(f"Percentage of titles with neutral sentiment: {len(list(filter(lambda x: x[1] == 0, res))) / len(res)*100:.2f}%")
print()
print(f"Percentage of titles with comparison degrees: {len(list(filter(lambda x: len(x[2])> 0, res))) / len(res)*100:.2f}%")


# len(list(filter(lambda x: len(x[0])> 0, res))) / len(res)

Percentage of titles with NER: 78.24%

Percentage of titles with positive sentiment: 37.54%
Percentage of titles with negative sentiment: 22.58%
Percentage of titles with neutral sentiment: 39.88%

Percentage of titles with comparison degrees: 4.92%
